In [31]:
import numpy as np
import pandas as pd
import cv2
import os
import pathlib
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import PIL
import time

In [26]:
class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.compat.v1.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.compat.v1.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        #print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

In [27]:
model_dir = "D:\\RESEARCH\\Obscene-Video-Classifier\\HumanDetectionModel"
model_path = os.path.join(model_dir,"faster_rcnn_inception_v2_coco_2018_01_28\\frozen_inference_graph.pb")
odapi = DetectorAPI(path_to_ckpt=model_path)
threshold = 0.7

In [9]:
p_path = "D:\\RESEARCH\\Obscene-Video-Classifier\\Data\\ExtractedDataFromVideos\\video_data\\train\\porn"
np_path = "D:\\RESEARCH\\Obscene-Video-Classifier\\Data\\ExtractedDataFromVideos\\video_data\\train\\non_porn"

In [12]:
p_folder_list = [os.path.join(p_path,x) for x in os.listdir(p_path)]
np_folder_list = [os.path.join(np_path,x) for x in os.listdir(np_path)]

In [41]:
p_frames_list = []
np_frames_list = []
        
for folder in p_folder_list:
    all_frames_add = [os.path.join(folder,x) for x in os.listdir(folder)]
    frames_list = []
    try_count = 0
    while(len(frames_list) >= 16):
        frames = random.choices(all_frames_add,k = 1)
        img = cv2.imread(frames[0])
        img = cv2.resize(img, (600, 400))
        boxes, scores, classes, num = odapi.processFrame(img)

        thresh_count = 0
        for i in range(len(boxes)):
            # Class 1 represents human
            if classes[i] == 1 and scores[i] > threshold:
                thresh_count += 1
                box = boxes[i]
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)
        if try_count>60:
            break
        if thresh_count== 0 :
            try_count += 1
            continue
        else:
            frames_list.append(frames)
    for frame in frames_list: 
        p_frames_list.append(frame[0])        
        
for folder in np_folder_list:
    all_frames_add = [os.path.join(folder,x) for x in os.listdir(folder)]
    frames = random.choices(all_frames_add,k = 16)
    for frame in frames: 
        np_frames_list.append(frame)        

In [38]:
train_save_path = "D:\\testing\\train"
val_save_path = "D:\\testing\\val"
test_save_path = "D:\\testing\\test"
#Total frames will be around 32000
# 16000 Frames - porn 16000 Frames - non porn
# 22000 Frames - Training 5000- Validation 5000- Test
# 0-11000 - train_index 11000-13500 - validation_index 13500- 16000 - test_index
for i in range(len(p_frames_list)):
    if i < int(len(p_frames_list)*0.6875):
        img = cv2.imread(p_frames_list[i])
        img = cv2.resize(img,(224,224)) 
        cv2.imwrite(os.path.join(train_save_path,"porn")+"\\"+str(i)+".jpg",img)
    elif i > int(len(p_frames_list)*0.6875) and i < int(len(p_frames_list)*0.84375):
        img = cv2.imread(p_frames_list[i])
        img = cv2.resize(img,(224,224)) 
        cv2.imwrite(os.path.join(val_save_path,"porn")+"\\"+str(i)+".jpg",img)    
    elif i > int(len(p_frames_list)*0.84375):
        img = cv2.imread(p_frames_list[i])
        img = cv2.resize(img,(224,224)) 
        cv2.imwrite(os.path.join(test_save_path,"porn")+"\\"+str(i)+".jpg",img)    

for i in range(len(np_frames_list)):
    if i < int(len(np_frames_list)*0.6875):
        img = cv2.imread(np_frames_list[i])
        img = cv2.resize(img,(224,224)) 
        cv2.imwrite(os.path.join(train_save_path,"non_porn")+"\\"+str(i)+".jpg",img)
    elif i > int(len(np_frames_list)*0.6875) and i < int(len(p_frames_list)*0.84375):
        img = cv2.imread(np_frames_list[i])
        img = cv2.resize(img,(224,224)) 
        cv2.imwrite(os.path.join(val_save_path,"non_porn")+"\\"+str(i)+".jpg",img)    
    elif i > int(len(np_frames_list)*0.84375):
        img = cv2.imread(np_frames_list[i])
        img = cv2.resize(img,(224,224)) 
        cv2.imwrite(os.path.join(test_save_path,"non_porn")+"\\"+str(i)+".jpg",img)            